In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
import torch

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
torch.cuda.is_available()
torch.cuda.current_device()

0

# Load dataset

In [3]:
df = pd.read_csv("../datasets/translated_dataframe.csv")
df.head()

,title,subtitle,description,keyword 1,keyword 2,keyword 3,keyword 4,keyword 5
0,Eye Gaze,Simulated and real datasets of eyes looking in...,# Context\nThe main reason for making this dat...,искусство и развлечения,земля и природа,социальные науки,изображение,глаза и зрение
1,Military Aircraft Detection Dataset,military aircraft images with aircraft type an...,## Overview\nThis dataset is designed for obje...,искусство и развлечения,военные,авиация,компьютерное зрение,классификация
2,Bhagavad Gita Dataset,All verses in Sanskrit with their Hindi and En...,#Context\nThe Bhagavad Gita (Sanskrit: भगवद् ग...,религия и системы верований,лингвистика,nlp,текст,перевод
3,Bin Baz Fatwas,Main Source: https://github.com/Alsarmad/binba...,"**Dataset Description**\nThe ""Fatwaas from Bin...",религия и системы верований,nlp,текст,генерация текста-в-текст,арабский
4,Nepali Cheers Liquor store product details,Alcoholic Beverages sold in one of a online li...,Data scraped from Nepali Online Liquor selling...,алкоголь,python,непали,NaN,NaN


In [4]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['title', 'subtitle', 'description', 'keyword 1', 'keyword 2', 'keyword 3', 'keyword 4', 'keyword 5'],
    num_rows: 4033
})

# Setup dataset

In [5]:
MODEL_NAME = "IlyaGusev/saiga_llama3_8b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token, tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('<|begin_of_text|>', '<|eot_id|>')

In [6]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_new_tokens": 1536,
  "pad_token_id": 128000,
  "repetition_penalty": 1.12,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [7]:
def gen_batches_train():
    for sample in iter(ds):
        # Extract instruction and input from the sample
        system_prompt = """Задача: Как ведущий редактор компании мирового уровня, вам поручено выполнить ключевую задачу: провести детальный анализ предоставленного текста и выделить ключевые слова и фразы, которые наиболее точно отражают его содержание. Эти ключевые слова и фразы будут использованы для создания тегов, которые должны быть релевантными, конкретными и краткими. Теги помогут пользователям быстро понять основные темы и характеристики текста.

Контекст и мотивация: Успешное выполнение этой задачи имеет решающее значение для будущего компании, в которой вы работаете уже 20 лет. От вашей работы зависит не только имидж компании, но и ваша собственная карьера, включая премии и потенциальную долю в компании. Поэтому крайне важно подойти к задаче с максимальной тщательностью и профессионализмом.

Инструкция по выполнению задачи:

Анализ текста: Внимательно прочитайте текст, чтобы понять его основной смысл, контекст и основные идеи.

Выделение ключевых слов и фраз: Определите наиболее важные слова и фразы, которые наиболее точно отражают содержание текста. Обратите внимание на уникальные термины и фразы, которые чётко описывают темы и характеристики.

Создание тегов: На основе выделенных ключевых слов и фраз сформулируйте теги. Теги должны быть:

Релевантными: Полностью отражать содержание текста.
Конкретными: Учитывать специфические аспекты, а не общие термины.
Краткими: Один-два слова для обеспечения лёгкости восприятия.
Примечания:

Старайтесь избегать общих слов и фраз. Выбирайте наиболее точные термины.
Учитывайте, что теги должны быть понятны широкой аудитории и точно отражать основные идеи текста.
Текст для анализа:
"""
        input_text = f"Заголовок: {sample['title']}"
        if sample['subtitle'] != '':
            input_text += f"\nПодзаголовок: {sample['subtitle']}"
        input_text += f"\nОписание: {sample['description']}"
        out_text = f"{sample['keyword 1']}"
        if sample['keyword 2'] != '':
            out_text += f", {sample['keyword 2']}"
        if sample['keyword 3'] != '':
            out_text += f", {sample['keyword 3']}"
        if sample['keyword 4'] != '':
            out_text += f", {sample['keyword 4']}"
        if sample['keyword 5'] != '':
            out_text += f", {sample['keyword 5']}"
            
        formatted_prompt = tokenizer.apply_chat_template([{
                "role": "system",
                "content": system_prompt
            }, {
                "role": "user",
                "content": input_text
            }, {
                "role": "assistant",
                "content": out_text
            }], tokenize=False, add_generation_prompt=False)
        
        yield {'text': formatted_prompt}

next(gen_batches_train())

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nЗадача: Как ведущий редактор компании мирового уровня, вам поручено выполнить ключевую задачу: провести детальный анализ предоставленного текста и выделить ключевые слова и фразы, которые наиболее точно отражают его содержание. Эти ключевые слова и фразы будут использованы для создания тегов, которые должны быть релевантными, конкретными и краткими. Теги помогут пользователям быстро понять основные темы и характеристики текста.\n\nКонтекст и мотивация: Успешное выполнение этой задачи имеет решающее значение для будущего компании, в которой вы работаете уже 20 лет. От вашей работы зависит не только имидж компании, но и ваша собственная карьера, включая премии и потенциальную долю в компании. Поэтому крайне важно подойти к задаче с максимальной тщательностью и профессионализмом.\n\nИнструкция по выполнению задачи:\n\nАнализ текста: Внимательно прочитайте текст, чтобы понять его основной смысл, контекст и основные ид

# Prepare model

In [8]:
model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, 
        device_map={"": 0}, 
        torch_dtype=torch.bfloat16,
    )

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


In [9]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type=TaskType.CAUSAL_LM, 
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    )

In [10]:
# tokenizer.pad_token = tokenizer.eos_token

# Training

In [11]:
training_arguments = TrainingArguments(
    output_dir='./saiga_results_ru',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="adamw_torch",
    save_steps=100,
    logging_steps=5,
    learning_rate=3e-4,
    fp16=False,
    bf16=True,
    num_train_epochs=1,
    report_to="none"
)

train_gen = Dataset.from_generator(gen_batches_train)
tokenizer.padding_side = "right"

In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_gen,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [13]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.578600
10,0.828000
15,0.614000
20,0.952200
25,0.740500
30,0.734100
35,0.856600
40,0.815900
45,0.792900
50,0.842100


TrainOutput(global_step=4033, training_loss=0.7216523939523335, metrics={'train_runtime': 2569.6803, 'train_samples_per_second': 1.569, 'train_steps_per_second': 1.569, 'total_flos': 1.516021345490043e+17, 'train_loss': 0.7216523939523335, 'epoch': 1.0})

In [14]:
peft_model_id="./saiga_lora2_ru"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('./saiga_lora2_ru/tokenizer_config.json',
 './saiga_lora2_ru/special_tokens_map.json',
 './saiga_lora2_ru/tokenizer.json')

In [ ]:
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto",torch_dtype=torch.bfloat16)

model = PeftModel.from_pretrained(model, model_id=peft_model_id, config=peft_config)

model = model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.11s/it]


In [ ]:
def test(question):
    system_prompt = """Задача: Как ведущий редактор компании мирового уровня, вам поручено выполнить ключевую задачу: провести детальный анализ предоставленного текста и выделить ключевые слова и фразы, которые наиболее точно отражают его содержание. Эти ключевые слова и фразы будут использованы для создания тегов, которые должны быть релевантными, конкретными и краткими. Теги помогут пользователям быстро понять основные темы и характеристики текста.

Контекст и мотивация: Успешное выполнение этой задачи имеет решающее значение для будущего компании, в которой вы работаете уже 20 лет. От вашей работы зависит не только имидж компании, но и ваша собственная карьера, включая премии и потенциальную долю в компании. Поэтому крайне важно подойти к задаче с максимальной тщательностью и профессионализмом.

Инструкция по выполнению задачи:

Анализ текста: Внимательно прочитайте текст, чтобы понять его основной смысл, контекст и основные идеи.

Выделение ключевых слов и фраз: Определите наиболее важные слова и фразы, которые наиболее точно отражают содержание текста. Обратите внимание на уникальные термины и фразы, которые чётко описывают темы и характеристики.

Создание тегов: На основе выделенных ключевых слов и фраз сформулируйте теги. Теги должны быть:

Релевантными: Полностью отражать содержание текста.
Конкретными: Учитывать специфические аспекты, а не общие термины.
Краткими: Один-два слова для обеспечения лёгкости восприятия.
Примечания:

Старайтесь избегать общих слов и фраз. Выбирайте наиболее точные термины.
Учитывайте, что теги должны быть понятны широкой аудитории и точно отражать основные идеи текста.
Текст для анализа:
"""
    input_text = f"Заголовок: {question['title']}"
    if question['subtitle'] != '':
        input_text += f"\nПодзаголовок: {question['subtitle']}"
    input_text += f"\nОписание: {question['description']}"
    formatted_prompt = tokenizer.apply_chat_template([{
            "role": "system",
            "content": system_prompt
        }, {
            "role": "user",
            "content": input_text
        }], tokenize=False, add_generation_prompt=True)
    
    print("INPUT:")
    print(formatted_prompt)

    model_inputs = tokenizer([formatted_prompt], return_tensors="pt").to('cuda')

    generated_ids = model.generate(
        input_ids=model_inputs.input_ids,
        max_new_tokens=32,
        eos_token_id=tokenizer.encode('<|eot_id|>')[0],
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]

    print("\nOUTPUT:")
    print(response)

In [ ]:
#from dataset
test(df.sample(1).iloc[0])

In [ ]:
# trainer.push_to_hub("XaPoHbomj/saiga-llama3-8b-tags-tuned")